In [ ]:
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Patch

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import Dense, Input
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from os import listdir


In [ ]:
import time
import ROOT
import numpy as np
import pandas as pd
import root_pandas as rpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from root_pandas import read_root
from matplotlib import rc
from numpy import inf


import sys,pandas as pd, matplotlib , matplotlib.pyplot as plt, matplotlib.lines , numpy as np, math, pylab
#import ROOT
#from ROOT import TFile

import root_pandas
%matplotlib inline

rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

In [ ]:
# Global plot settings
from matplotlib import rc
import matplotlib.font_manager

rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

In [ ]:
# Define default plot styles
plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': True
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': True
}

plot_style_2 = {'alpha': 0.5, 'density': True}

In [ ]:
# Check Versions
print(tf.__version__)  # 1.15.0
print(keras.__version__)  # 2.2.4

## Obtain datasets and simulations

In [ ]:
data_hadron = pd.DataFrame() 
paths = ['/home/miguel/EG2_DATA/andres/D2_pb3/']
paths += ['/home/miguel/EG2_DATA/andres/D2_pb2/']
paths += ['/home/miguel/EG2_DATA/andres/D2_pb1/']

print(paths)
for path in paths:
    print (path)
    Files = listdir(path)
    Files.sort()
    for name in Files:
        filename = path+name
        print(filename)
        temp = root_pandas.read_root(filename, "ntuple_sim")
        data_hadron = pd.concat([data_hadron, temp])
        

print('Number of entries in datafarame is',data_hadron.shape[0])

In [ ]:
mc_hadron = data_hadron
print(len(mc_hadron))

In [ ]:
#path = '/home/miguel/EG2_DATA/sebastian/'
#data_electrons = root_pandas.read_root(path+'C.root', "ntuple_data_electrons")
#data_hadron = root_pandas.read_root(path+'C.root', "ntuple_data")

In [ ]:
path = '/home/miguel/EG2_DATA/andres/GetSimpleTuple_data/Pb/'
data_electrons = root_pandas.read_root(path+'Pb.root', "ntuple_e")
data_hadron = root_pandas.read_root(path+'Pb.root', "ntuple_data")

In [ ]:
#path = '/home/miguel/EG2_DATA/andres/Pb1/'
path = '/home/miguel/EG2_DATA/andres/D2_pb1/'
mc_electrons = root_pandas.read_root(path+'D_1.root', "ntuple_e")
mc_hadron = root_pandas.read_root(path+'D_1.root', "ntuple_sim")

In [ ]:
mc_hadron['pass_reco'] = np.where(mc_hadron['Pt2']>0, 1, 0)
mc_hadron['pass_truth'] = np.where(mc_hadron['mc_Pt2']>0, 1, 0)
data_hadron['pass_reco'] = np.where(data_hadron['Pt2']>0, 1, 0)

In [ ]:
data_hadron['pid'] = data_hadron['PID']


## Define cuts

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print (text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut))
    return cutDataframe

def applyCutsHadron(df,isMC=False):
    temp = df
    ##temp.eval('inelasticity = Nu/5.014', inplace=True)
    temp.eval('P = sqrt(Px*Px+Py*Py+Pz*Pz)',inplace=True)

    if(isMC):
        temp = applyCut(temp,'pass_truth>0',' pass_truth>0')
        temp = applyCut(temp,'mc_pid==211',' mc_pid==211')
        
        temp.eval('mc_P = sqrt(mc_Px*mc_Px+mc_Py*mc_Py + mc_Pz*mc_Pz)',inplace=True)
        temp.eval('mc_E = sqrt(mc_P*mc_P+0.1396*0.1396)', inplace=True)
    else:
        temp = applyCut(temp, 'abs(deltaZ)<3', 'abs(deltaZ)<3 [cm]')
        #temp = applyCut(temp, 'abs(YEC)<1.4',   'abs(YEC)<1.4 [cm]')
    
    #temp = applyCut(temp, 'Q2 > 1 && W > 2 && vyec < 1.4 && vyec > -1.4')
    temp = applyCut(temp, 'pass_reco==0 or pid==211', 'pid==211')
    temp = applyCut(temp, 'pass_reco==0 or Q2>1.0', 'Q2 >1.0 GeV2')
    temp = applyCut(temp, 'pass_reco==0 or Nu>2.5', 'Nu >2.5 GeV')
    temp = applyCut(temp, 'pass_reco==0 or Yb<0.85', ' Yb<0.85')
        #temp = applyCut(temp, 'TargType==2',   'TargType==2') ##Target 1 is deuterium, 2 is solid

    return temp

## Apply cuts

In [ ]:
print('Selecting MC events\n')
mc_hadron = applyCutsHadron(mc_hadron,isMC=True)
print('Selecting data events\n')
data_hadron= applyCutsHadron(data_hadron,isMC=False)

## Target type ==1 deuterium and Targtype==2

In [ ]:
data_hadron = data_hadron.query('TargType==1') 

## Count the non zero pass_reco and pass_truth

In [ ]:
nevents = int(0.5e6)
#data = data_hadron.sample(n=nevents)
#mc   = mc_hadron.sample(n=nevents)
print
data = data_hadron.sample(n=30000)
mc   = mc_hadron.sample(n=800000)

In [ ]:
## You want reasonable sample of MC, so e.g x 10 the data size
print(len(data))
print(len(mc.query('pass_reco==1')))

In [ ]:

##Hadron momentum
Px_0_G = np.array(mc['mc_Px'])
Px_0_S =np.array(mc['Px'])
Px_unknown_S = np.array(data['Px'])

Py_0_G = np.array(mc['mc_Py'])
Py_0_S =np.array(mc['Py'])
Py_unknown_S = np.array(data['Py'])

Pz_0_G = np.array(mc['mc_Pz'])
Pz_0_S = np.array(mc['Pz'])
Pz_unknown_S = np.array(data['Pz'])

Pid_0_G = np.array(mc['mc_pid'])/100.0
Pid_0_S = np.array(mc['pid'])/100.0
Pid_unknown_S = np.array(data['pid'])/100.0

Q2_0_G = np.array(mc['mc_Q2'])
Q2_0_S = np.array(mc['Q2'])
Q2_unknown_S = np.array(data['Q2'])

Nu_0_G = np.array(mc['mc_Nu'])
Nu_0_S = np.array(mc['Nu'])
Nu_unknown_S = np.array(data['Nu'])

PhiPQ_0_G = np.array(mc['mc_PhiPQ'])/100.0
PhiPQ_0_S = np.array(mc['PhiPQ'])/100.0
PhiPQ_unknown_S = np.array(data['PhiPQ'])/100.0

Zh_0_G = np.array(mc['mc_Zh'])
Zh_0_S =np.array(mc['Zh'])
Zh_unknown_S = np.array(data['Zh'])

Pt2_0_G = np.array(mc['mc_Pt2'])
Pt2_0_S = np.array(mc['Pt2'])
Pt2_unknown_S = np.array(data['Pt2'])


In [ ]:
plt.hist(mc.query('pass_truth==1 and pass_reco==1')['PhiPQ'])

In [ ]:
theta0_G=np.c_[(Px_0_G,Py_0_G,Pz_0_G,Q2_0_G,PhiPQ_0_G)]
theta0_S=np.c_[(Px_0_S,Py_0_S,Pz_0_S,Q2_0_S,PhiPQ_0_S)]
theta_unknown_S=np.c_[(Px_unknown_S,Py_unknown_S,Pz_unknown_S,Q2_unknown_S, PhiPQ_unknown_S)]


In [ ]:
theta0_G=np.c_[(Px_0_G,Py_0_G,Pz_0_G,Q2_0_G,Nu_0_G,PhiPQ_0_G,Pt2_0_G)]
theta0_S=np.c_[(Px_0_S,Py_0_S,Pz_0_S,Q2_0_S,Nu_0_S,PhiPQ_0_S,Pt2_0_S)]
theta_unknown_S=np.c_[(Px_unknown_S,Py_unknown_S,Pz_unknown_S,Q2_unknown_S,Nu_unknown_S,PhiPQ_unknown_S,Pt2_unknown_S)]


# OmniFold Gaussian Toy Example

## Setup the Datasets

In [ ]:
bins = np.linspace(-2,5, 101)

fig, ax = plt.subplots(1,
                       2,
                       figsize=(12, 5),
                       constrained_layout=True,
                       sharey=True)

ax[0].set_xlabel('$x_0$')
ax[0].set_ylabel('Events per bin (normalized)')
ax[0].hist(theta0_G[theta0_G[:,0]!=-10][:,0], bins=bins, **plot_style_2, label='Generation')
ax[0].hist(theta0_S[theta0_S[:,0]!=-10][:,0], bins=bins, **plot_style_2, label='Simulation')
legend = ax[0].legend(
    loc='upper left',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

ax[1].set_xlabel('$x_0$')
ax[1].hist(theta_unknown_S[theta_unknown_S[:,0]!=-10][:,0], bins=bins, **plot_style_2, label='Detector')
legend = ax[1].legend(
    loc='upper left',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

fig.show()

#bins = np.linspace(-180,180, 101)
#bins = np.linspace(0,2, 101)
bins = np.linspace(-3,3,4)
fig, ax = plt.subplots(1,
                       2,
                       figsize=(12, 5),
                       constrained_layout=True,
                       sharey=True)

ax[0].set_xlabel('$x_1$')
ax[0].set_ylabel('Events per bin (normalized)')
ax[0].hist(theta0_G[theta0_G[:,0]!=-10][:,-1], bins=bins, **plot_style_2, label='Generation')
ax[0].hist(theta0_S[theta0_S[:,0]!=-10][:,-1], bins=bins, **plot_style_2, label='Simulation')
legend = ax[0].legend(
    loc='upper left',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

ax[1].set_xlabel('$x_1$')
ax[1].hist(theta_unknown_S[theta_unknown_S[:,0]!=-10][:,-1], bins=bins, **plot_style_2, label='Detector')
legend = ax[1].legend(
    loc='upper left',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

fig.show()

## flag entries that do not pass reco (due to efficiency) or not pass truth (fakes)

In [ ]:
pass_reco = np.array(mc['pass_reco'])
pass_truth = np.array(mc['pass_truth'])
theta0_S[:,0][pass_reco==0] = -10
theta0_G[:,0][pass_truth==0] = -10

## Unfold Data

In [ ]:
# Input multiple observables as tuple or list (i.e. theta0_G = (obs_1, obs_2, ...))
"""    Arguments:
        num_observables: number of observables to 
                            simultaneously unfold (integer)
                            
        iterations: number of iterations (integer)
        
        theta0_G: tuple or list of nominal 
                    generation-level observables as Numpy arrays
                    
        theta0_S: tuple or list of nominal 
                    simulation-level observables as Numpy arrays
                    
        theta_unknown_S: tuple or list of "natural" 
                            (unknown) simulation-level observables 
                            to be unfolded as Numpy arrays
        
        Returns:
        - A Numpy array of weights to reweight distributions in 
        theta0_G to the unfolded distribution of theta_unknown_S
        
        - The model used to calculate those weights
"""


def multifold(num_observables, iterations, theta0_G, theta0_S,
              theta_unknown_S):
    
    
    theta0 = np.stack([theta0_G, theta0_S], axis=1)
    labels0 = np.zeros(len(theta0))
    theta_unknown = np.stack([theta_unknown_S, theta_unknown_S], axis=1)
    #labels_unknown = np.ones(len(theta_unknown))
    
    #xvals_1 = np.concatenate((theta0_S, theta_unknown_S))
    #yvals_1 = np.concatenate((labels0, labels_unknown))
    #xvals_2 = np.concatenate((theta0_G, theta0_G))
    #yvals_2 = np.concatenate((labels0, labels_unknown))
    
    
    #theta0_G = np.stack(theta0_G, axis=1)
    #theta0_S = np.stack(theta0_S, axis=1)
    #theta_unknown_S = np.stack(theta_unknown_S, axis=1)

    #labels0 = np.zeros(len(theta0_G))
    labels1 = np.ones(len(theta0_G))
    labels_unknown = np.ones(len(theta_unknown_S))
    
    xvals_1 = np.concatenate((theta0_S, theta_unknown_S))
    yvals_1 = np.concatenate((labels0, labels_unknown))
    xvals_2 = np.concatenate((theta0_G, theta0_G))
    yvals_2 = np.concatenate((labels0, labels1))
    

    weights = np.empty(shape=(iterations, 2, len(theta0_G))) #this was 0
    # shape = (iteration, step, event)
    
    # initial iterative weights are ones

    weights_pull = np.ones(len(theta0_S))
    weights_push = np.ones(len(theta0_S))

    inputs = Input((num_observables, ))
     #inputs = Input((2, ))
    hidden_layer_1 = Dense(50, activation='relu')(inputs)
    hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
    outputs = Dense(1, activation='sigmoid')(hidden_layer_3)

    model = Model(inputs=inputs, outputs=outputs)

    earlystopping = EarlyStopping(patience=10,
                              verbose=1,
                              restore_best_weights=True)
    # from NN (DCTR) 
    def reweight(events):
        f = model.predict(events, batch_size=10000)
        weights = f / (1. - f)
        return np.squeeze(np.nan_to_num(weights))
    
    
    
    for i in range(iterations):
        print("\nITERATION: {}\n".format(i + 1))

        # STEP 1: classify Sim. (which is reweighted by weights_push) to Data
        # weights reweighted Sim. --> Data
        print("STEP 1\n")

        weights_1 = np.concatenate((weights_push, np.ones(len(theta_unknown_S))))
        # actual weights for Sim., ones for Data (not MC weights)

        X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(
            xvals_1, yvals_1, weights_1)

        #theta0_G[theta0_S[:,0]!=-10][:,1]
        model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
        model.fit(X_train_1[X_train_1[:,0]!=-10],
              Y_train_1[X_train_1[:,0]!=-10],
              sample_weight=w_train_1[X_train_1[:,0]!=-10],
              epochs=200,
              batch_size=10000,
              validation_data=(X_test_1[X_test_1[:,0]!=-10], Y_test_1[X_test_1[:,0]!=-10], w_test_1[X_test_1[:,0]!=-10]),
              callbacks=[earlystopping],
              verbose=1)

        weights_pull = weights_push * reweight(theta0_S)
        weights_pull[theta0_S[:,0]==-10] = 1. #these are events that don't pass reco; take the prior.
        weights[i, :1, :] = weights_pull

        # STEP 2: classify Gen. to reweighted Gen. (which is reweighted by weights_pull)
        # weights Gen. --> reweighted Gen.
        print("\nSTEP 2\n")

        weights_2 = np.concatenate((np.ones(len(theta0_G)), weights_pull))
        # ones for Gen. (not MC weights), actual weights for (reweighted) Gen.

        X_train_2, X_test_2, Y_train_2, Y_test_2, w_train_2, w_test_2 = train_test_split(
            xvals_2, yvals_2, weights_2)

        model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
        model.fit(X_train_2,
              Y_train_2,
              sample_weight=w_train_2,
              epochs=200,
              batch_size=10000,
              validation_data=(X_test_2, Y_test_2, w_test_2),
              callbacks=[earlystopping],
              verbose=1)

        weights_push = reweight(theta0_G)
        weights[i, 1:2, :] = weights_push
    return weights, model

    

In [ ]:
K.clear_session()


In [ ]:
iterations = 4
num_observables= 6
# simple / 100. standardization s.t. data is of order ~1

weights, _ = multifold(num_observables=num_observables,
                       iterations=iterations,
                       theta0_G=theta0_G,
                       theta0_S=theta0_S,
                       theta_unknown_S= theta_unknown_S)

## Plot data and MC reweighted

In [ ]:
## This is what in Ben's notebook. 
## The _S is RECO level. 
## You require that the RECO dataframe has a valid pass reco.
## The G is the truth level
## You required that the truth level has pass_truth valid.

In [ ]:
for i in range(len(weights)):

    print("ITERATION: {}".format(i + 1))
    bins = np.linspace(0.0, 4.0, 40)
    
    fig, ax = plt.subplots(3,
                           3,
                           figsize=(16, 16),
                           constrained_layout=True)
    ax[0,0].set_xlabel(r'$Q^{2}$')
    ax[0,0].set_ylabel('Events per bin (Normalized)')
    ax[0,0].hist(Q2_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[0,0].hist(Q2_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[0,0].hist(Q2_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    ax[0,0].legend(frameon=False,loc='best')
    
    
    bins = np.linspace(0.0, 5, 20)

    ax[0,1].set_xlabel(r'$\nu$')
    ax[0,1].set_ylabel('Events per bin (Normalized)')
    ax[0,1].hist(Nu_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[0,1].hist(Nu_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[0,1].hist(Nu_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    
    bins = np.linspace(-1.5, 1.5, 50)

    ax[0,2].set_xlabel(r'$P_{x}$')
    ax[0,2].set_ylabel('Events per bin (Normalized)')
    ax[0,2].hist(Px_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[0,2].hist(Px_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[0,2].hist(Px_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    bins = np.linspace(-1.5, 1.5, 20)

    ax[1,0].set_xlabel(r'$P_{y}$')
    ax[1,0].set_ylabel('Events per bin (Normalized)')
    ax[1,0].hist(Py_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[1,0].hist(Py_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[1,0].hist(Py_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    bins = np.linspace(-0.5, 3.0, 50)

    ax[1,1].set_xlabel(r'$P_{z}$')
    ax[1,1].set_ylabel('Events per bin (Normalized)')
    ax[1,1].hist(Pz_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[1,1].hist(Pz_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[1,1].hist(Pz_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    bins = np.linspace(-1.8, 1.8, 100)

    ax[1,2].set_xlabel(r'$\phi_{PQ}$')
    ax[1,2].set_ylabel('Events per bin (Normalized)')
    ax[1,2].hist(PhiPQ_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[1,2].hist(PhiPQ_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[1,2].hist(PhiPQ_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    bins = np.linspace(0, 1.2, 50)

    ax[2,0].set_xlabel(r'$z$')
    ax[2,0].set_ylabel('Events per bin (Normalized)')
    ax[2,0].hist(Zh_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[2,0].hist(Zh_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[2,0].hist(Zh_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    
    bins = np.linspace(0, 2, 50)

    ax[2,1].set_xlabel(r'$p_{T}^{2}$')
    ax[2,1].set_ylabel('Events per bin (Normalized)')
    ax[2,1].hist(Pt2_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[2,1].hist(Pt2_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[2,1].hist(Pt2_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    
    bins = np.linspace(-300, 300, 400)

    ax[2,2].set_xlabel('PID')
    ax[2,2].set_ylabel('Events per bin (Normalized)')
    ax[2,2].hist(Pid_0_S[theta0_S[:,0]!=0], bins=bins, label=r'MC .', **plot_style_2)
    ax[2,2].hist(Pid_0_S[theta0_S[:,0]!=0],
               bins=bins,
               weights=weights[i, 0, :][theta0_S[:,0]!=0],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[2,2].hist(Pid_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    

## Put weights obtained with the DNN back into the mc dataframe:

In [ ]:
mc['weight_reco'] = weights[-1, 0, :]
mc['weight_truth'] = weights[-1, 1, :]

## plot weights

In [ ]:
mc.query('pid==-211')['weight_reco']

In [ ]:
plt.hist(mc.query('mc_pid==+211')['weight_reco'],bins=100, density=True,alpha=0.1)
plt.hist(mc.query('mc_pid==-211')['weight_reco'],bins=100,density=True,alpha=0.2)

plt.show()

In [ ]:
plt.hist(mc.query('mc_pid==+211')['weight_reco'],range=(0,2.0),bins=100, density=True,alpha=0.1)
plt.hist(mc.query('mc_pid==-211')['weight_reco'],range=(0.,2.0),bins=100,density=True,alpha=0.2)

plt.show()

## Transform dataframe back into ROOT file 

In [ ]:
import root_pandas
from root_pandas import read_root
from root_pandas import to_root 

In [ ]:
to_root(mc,'Pb_weighted.root', key='ntuple_sim')


## Compute efficiency

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(16,5),sharey=True)

mask = np.logical_and(theta0_S[:,0]!=-10,theta0_G[:,0]!=-10)

bins = np.linspace(0,1.0,40)
axs[0].set_title('Non weighted')
axs[1].set_title('Weighted')
#axs[0].hist(Zh_0_G[theta0_S[:,0]==-10],alpha=0.3,bins=100, label='all generated')
axs[0].hist(Zh_0_G[theta0_G[:,0]!=-10],alpha=0.3,bins=bins, label='all generated')
axs[0].hist(Zh_0_G[mask], alpha=0.3,bins=bins, label='pass reco')
#axs[1].hist(Zh_0_G[theta0_S[:,0]==-10],alpha=0.3,bins=100,weights=weights[-1, 1, :][theta0_S[:,0]==-10],label='all generated')
axs[1].hist(Zh_0_G[theta0_G[:,0]!=-10],alpha=0.3,bins=bins,weights=weights[-1, 1, :][theta0_G[:,0]!=-10],label='all generated')
axs[1].hist(Zh_0_G[mask],alpha=0.3,bins=bins,weights=weights[-1, 1, :][mask],label='pass reco')


axs[0].set(xlabel=r"$z$", ylabel="Events")
axs[1].set(xlabel=r"$z$", ylabel="Events")
plt.legend()
plt.show()

In [ ]:
#numerator
mask = np.logical_and(theta0_S[:,0]!=-10,theta0_G[:,0]!=-10)
print (mask) 
num,x,_  = plt.hist(Zh_0_G[mask], weights=weights[-1, 1, :][mask],bins=20,label='numerator, weighted',alpha=0.3)
den,x,_ = plt.hist(Zh_0_G[theta0_G[:,0]!=-10], weights=weights[-1, 1, :][theta0_G[:,0]!=-10],bins=20,label='numerator, weighted',alpha=0.3)
x= (x[1:] + x[:-1])/2.0

#plt.hist(theta0_G[theta0_S[:,0]!=-10][:,1])
plt.show()
eff_weighted= np.true_divide(num,den)
print(sum(num))
print(sum(den))
print(sum(num)/sum(den))
#plt.legend()

## Plot efficiency with no weighting (i.e. out of the box MC)

In [ ]:
#numerator
mask = np.logical_and(theta0_S[:,0]!=-10,theta0_G[:,0]!=-10)
print (mask) 
num,x,_  = plt.hist(Zh_0_G[mask], bins=20,label='numerator, weighted',alpha=0.3)
den,x,_ = plt.hist(Zh_0_G[theta0_G[:,0]!=-10], bins=20,label='numerator, weighted',alpha=0.3)
x= (x[1:] + x[:-1])/2.0

#plt.hist(theta0_G[theta0_S[:,0]!=-10][:,1])
plt.show()
eff= np.true_divide(num,den)
print(sum(num))
print(sum(den))
print(sum(num)/sum(den))
#plt.legend()

In [ ]:
plt.plot(x,eff,label='MC')
plt.plot(x,eff_weighted, label='MC weighted')

plt.ylabel('Acceptance')
plt.xlabel('z')
plt.legend(loc='best',frameon=False)
plt.ylim([0.,0.2])
plt.xlim([0.,1.0])

## Plot the unfolded distributions

In [ ]:
bins = np.linspace(-2, 2, 101)

for i in range(1):
    print("ITERATION {}:".format(i + 1))
    plt.figure(figsize=(10, 5))
    #fig, (ax1, ax2) = plt.subplots(1, 2)
    
    gs = plt.GridSpec(1, 2)
    gs.update(left=0.08, right=0.925,
              top=0.95, bottom=0.05,
              hspace=0.3, wspace=0.5)

    ax1 = plt.subplot(gs[0, 0])
    ax2 = plt.subplot(gs[0, 1])     
    
    hist0 = ax1.hist(theta0_G[theta0_G[:,0]!=-10][:,0],
                     bins=bins,
                     **plot_style_2,label='Gen')

    hist1 = ax1.hist(
        theta0_G[theta0_G[:,0]!=-10][:,0],
        bins=bins,
        label='Gen weighted',
        weights=weights[i, 1, :][theta0_G[:,0]!=-10],
        **plot_style_1)

    ax1.legend(frameon=False)
    ax1.set(xlabel=r"$x_{1,G}$", ylabel="Events per bin (normalized)")
    #ax1.set_ylim([0,0.6])
    bins = np.linspace(-2, 5, 101)

    hist0 = ax2.hist(theta0_G[theta0_G[:,0]!=-10][:,2],
                     bins=bins,
                     **plot_style_2)

    hist1 = ax2.hist(
        theta0_G[theta0_G[:,0]!=-10][:,2],
        bins=bins,
        weights=weights[i, 1, :][theta0_G[:,0]!=-10],
        **plot_style_1)

    ax2.legend(frameon=False)
    ax2.set(xlabel=r"$x_{2,G}$", ylabel="Events per bin (normalized)")
    #ax2.set_ylim([0,0.6])